In [ ]:
# 1. INSTALL & SETUP
# Install the official libtorrent package (might be older version 1.1.x, but we will adapt)
!pip install libtorrent
!curl https://rclone.org/install.sh | sudo bash

import libtorrent as lt
import time
import os
import shutil
import sys

print(f"Libtorrent Version: {lt.version}")

def save_resume_data(handle, save_dir):
    if not handle.need_save_resume_data():
        return
    
    handle.save_resume_data()
    # Wait for alert
    limit = 0
    while limit < 30:
        alerts = ses.pop_alerts()
        for a in alerts:
            if isinstance(a, lt.save_resume_data_alert):
                path = os.path.join(save_dir, f"{handle.info_hash()}.fastresume")
                with open(path, 'wb') as f:
                    f.write(lt.bencode(a.resume_data))
                print(f"\r[Checkpointed] Saved resume data to {path}", end='')
                return
        time.sleep(0.1)
        limit += 1

print("Libraries Installed. Ready to Mount.")

In [ ]:
# 2. MOUNT GOOGLE DRIVE (PERSISTENT)
from google.colab import drive
# 1. Mount Standard Drive to save config
drive.mount('/content/drive')

RCLONE_CONF_PATH = '/root/.config/rclone/rclone.conf'
DRIVE_CONF_BACKUP = '/content/drive/MyDrive/rclone.conf'

if not os.path.exists('/root/.config/rclone'):
    os.makedirs('/root/.config/rclone')

# 2. Check for existing config in Drive
if os.path.exists(DRIVE_CONF_BACKUP):
    print("Found existing Rclone config in Drive! Loading...")
    shutil.copy(DRIVE_CONF_BACKUP, RCLONE_CONF_PATH)
else:
    print("--- First Time Rclone Setup ---")
    print("1. Run this cell.")
    print("2. It will ask for a config. Type 'n' (New remote).")
    print("3. Name: 'gdrive'")
    print("4. Storage: 'drive' (Google Drive)")
    print("5. Client ID/Secret: Leave blank (Press Enter)")
    print("6. Scope: '1' (Full access)")
    print("7. Root Folder: Leave blank")
    print("8. Service Account: Leave blank")
    print("9. Edit advanced config: 'n'")
    print("10. Use web browser: 'n' (Since we are headless)")
    print("11. It will give you a link. Click it, login, copy code, paste it back.")
    print("12. Shared Drive: 'n'")
    print("13. Keep this remote: 'y'")
    print("14. Quit: 'q'")
    
    !rclone config
    
    # Backup config to Drive for next time
    if os.path.exists(RCLONE_CONF_PATH):
        print("Saving config to Drive for future use...")
        shutil.copy(RCLONE_CONF_PATH, DRIVE_CONF_BACKUP)

if not os.path.exists('/content/drive_rclone'):
    os.makedirs('/content/drive_rclone')
    print("Mounting Drive (Rclone)...")
    # Mount with VFS Cache for stability
    get_ipython().system_raw(
        'nohup rclone mount gdrive: /content/drive_rclone '
        '--vfs-cache-mode writes '
        '--vfs-cache-max-size 10G '
        '--no-modtime '
        '--daemon &'
    )
    time.sleep(3)

print("Drive Mounted at /content/drive_rclone")

In [ ]:
# 3. DOWNLOADER (Universal Compatibility)
SAVE_PATH = "/content/drive_rclone/Torrent"
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

ses = lt.session()
ses.listen_on(6881, 6891)

# --- POLYFILL: Handle both Modern (2.0+) and Legacy (1.1.x) Libtorrent APIs ---
if hasattr(lt, 'settings_pack') and hasattr(ses, 'apply_settings'):
    print("Using Modern API (settings_pack)")
    pack = lt.settings_pack()
    pack.set_int('cache_size', 65536)
    pack.set_bool('enable_os_cache', True)
    # pack.set_str('listen_interfaces', '0.0.0.0:6881,0.0.0.0:6891') # Optional in 2.0
    ses.apply_settings(pack)
elif hasattr(ses, 'set_settings'):
    print("Using Legacy API (set_settings)")
    # Legacy dictionary-based settings
    settings = {
        'cache_size': 65536,
        'enable_os_cache': True,
        'active_downloads': -1,
        'active_seeds': -1
    }
    ses.set_settings(settings)
else:
    print("Warning: Could not configure settings (No supported API found). Using defaults.")
# -----------------------------------------------------------------------------

link = input("Enter Magnet Link: ")
params = lt.parse_magnet_uri(link)
params.save_path = SAVE_PATH

# Handle storage_mode (Deprecated in 2.0, required in 1.1)
if hasattr(lt, 'storage_mode_t'):
    params.storage_mode = lt.storage_mode_t(2)

# Check for Resume Data
info_hash = str(params.info_hash)
resume_file = os.path.join(SAVE_PATH, f"{info_hash}.fastresume")

if os.path.exists(resume_file):
    print("Found resume data! Loading...")
    with open(resume_file, 'rb') as f:
        params.resume_data = f.read()

handle = ses.add_torrent(params)
handle.set_sequential_download(True)

print("Metadata Downloading...")
while not handle.has_metadata():
    time.sleep(1)

# File Selection
info = handle.get_torrent_info()
files = info.files()
print(f"\nTorrent: {info.name()}")
print("Files:")
for i in range(files.num_files()):
    print(f"[{i}] {files.file_path(i)} ({files.file_size(i) / 1024 / 1024:.2f} MB)")

selection = input("\nEnter file indices to download (comma separated, e.g. '0,2') or 'all': ")
if selection.lower() != 'all':
    indices = [int(x.strip()) for x in selection.split(',') if x.strip().isdigit()]
    priorities = [0] * files.num_files()
    for idx in indices:
        if 0 <= idx < files.num_files():
            priorities[idx] = 4 # 4 = Default priority
    handle.prioritize_files(priorities)
    print(f"Selected {len(indices)} files.")
else:
    print("Downloading ALL files.")

print(f"Started: {handle.status().name}")

try:
    while not handle.status().is_seeding:
        s = handle.status()
        # State string mapping (Safe for both versions)
        state_str = str(s.state)
        
        # Try to get readable state if possible
        if s.state == lt.torrent_status.checking_files:
            print(f'\rChecking: {s.progress * 100:.2f}% complete', end=' ')
        else:
            print(f'\r{s.progress * 100:.2f}% | {s.download_rate / 1000:.1f} kB/s | Peers: {s.num_peers} | State: {state_str}', end=' ')
        
        # Auto-checkpoint every minute
        if int(time.time()) % 60 == 0:
            save_resume_data(handle, SAVE_PATH)
            
        time.sleep(1)
    print("\nDownload Complete!")
    # Cleanup resume file on success
    if os.path.exists(resume_file):
        os.remove(resume_file)

except KeyboardInterrupt:
    print("\nStopping... Saving Resume Data...")
    save_resume_data(handle, SAVE_PATH)
    print("\nSaved. Safe to close.")